In [1]:
from core import pds as pds
import numpy as np
import pandas as pd

In [2]:
# pds frame definition
psd1d_frame = pds.PDS1D_SingleSource()

In [4]:
# setup mesh
nx = 100
dx = 1
x = np.arange(0, nx * dx, dx)

d = 100
u_init = np.zeros(nx)
t0 = 0
lbc = 'Neumann'
rbc = 'Neumann'

In [5]:
# pass the parameters
psd1d_frame.set_mesh(x)
psd1d_frame.set_diffusivity(d)
psd1d_frame.set_initial(u_init)
psd1d_frame.set_bcs(lbc, rbc)
psd1d_frame.set_t0(t0)

In [10]:
# Test snapshot
psd1d_frame.solve(dt = 2, t_total= 200)
psd1d_frame.snapshot[:5,:5]

Time array generated using t_total.


array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])